#PostgreSQL Scripts
#Create Database
CREATE DATABASE flickr;
USE flickr;

#CREATE TABLES

CREATE TABLE group(
group_id text PRIMARY NOT NULL,
group_name text NOT NULL,
member_count INT NOT NULL,
pool_count INT NOT NULL,
topic_count INT NOT NULL
);

CREATE TABLE member(
member_id text PRIMARY KEY NOT NULL,
member_name text NOT NULL
);

CREATE TABLE member_type(
membertype_id INT PRIMARY KEY NOT NULL,
member_type text NOT NULL
);

CREATE TABLE group_member(
group_member_id SERIAL PRIMARY KEY NOT NULL,
group_id text REFERENCES group(group_id) NOT NULL,
member_id text REFERENCES member(member_id) NOT NULL,
membertype_id REFERENCES member_type(membertype_id) NOT NULL
);

CREATE TABLE group_admin(
group_admin_id SERIAL PRIMARY KEY NOT NULL,
group_id text REFERENCES group(group_id) NOT NULL,
admin_id text REFERENCES member(member_id) NOT NULL
);


#INSERT INTO TABLE--COPY FROM CSV
\copy group (group_id, group_name, member_count, pool_count, topic_count) from 'Nature_Group.csv' csv header;
\copy member (member_id, member_name) from 'UniqueMember.csv' csv header;
\copy group_member (group_id, member_id) from 'Group_Member.csv' csv header;
\copy group_admin (group_id, admin_id) from 'Group_Admin.csv' csv header;

In [ ]:
#CREATE TABLES FROM RAW DATA

#UniqueMember--data for table member

import glob
import os
import json as js

#Read json files for each group
file_list = glob.glob(os.path.join(r"flickr\member_data\MemberInfo_JSON", "*.json"))
memberlist = []

for file_path in file_list:
    with open(file_path) as file_input:
        data = js.load(file_input)
        memberlist.append(data)
        
#Look up member info in the loaded list
import pandas as pd
from nested_lookup import nested_lookup
for i in range(len(memberlist)):
    j = memberlist[i]
    member_id = pd.Series(nested_lookup('nsid', j))
    member_name = pd.Series(nested_lookup('realname', j))
    df_member = pd.DataFrame([member_id, member_name], index = ['member_id', 'member_name'])

df_member.to_csv('UniqueMember.csv')
#Duplicate member_id can be easily droped in csv to create a table to store unique member data

#Group membership--data for table group_member
#Still use the memberlist
#Look up member_id and member_type
import pandas as pd
from nested_lookup import nested_lookup
for i in range(len(memberlist)):
    j = memberlist[i]
    member_id = pd.Series(nested_lookup('nsid', j))
    member_type = pd.Series(nested_lookup('membertype', j))
    
#Assign group id in terms of group membership
#Extract group ids
group_id = []
for i in range(len(memberlist)):
    j = memberlist[i]
    k = j[0]
    group_id.append(k)

#Group membership list
membership = {}
for i in range(len(memberlist)):
    membership[group_id[i]] = nested_lookup('nsid', memberlist[i])

#A list of lists of member ids for each group
group_member_id = list(membership.values())

#Calculate length for each list--count of members for each group
count_of_member = []
for i in range(len(group_member_id)):
    j = group_member_id[i]
    k = len(j)
    count_of_member.append(k)
    
#Repeat group ids based on the count of members of each group
import numpy as np
repeat_group_id = np.repeat(group_id, count_of_member)

#Create dataframe and to csv
df_group_member = pd.DataFrame([repeat_group_id, member_id, member_type],  
                               index = ['groupid','memberid','membertype'])
df_group_member.to_csv('Group_Member.csv')

#Similar codes for creating other tables
